In [ ]:
import sys
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cmocean as cmo
import dask
import dask.distributed as dd
import dask.array as da
from dask import delayed

from time import sleep

sys.path.append('../')

from geometry import bedmachine

np.set_printoptions(precision=2)
%matplotlib notebook
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
ds = xr.open_dataset("../../data/BedMachineAntarctica_2020-07-15_v02.nc")
var = ds.thickness.values

In [ ]:
def roll(x):
    x = np.roll(x,1,axis=0)
    return x

@delayed
def droll(x):
    x = np.roll(x,1,axis=0)
    return x

In [ ]:
chunk = 400

In [ ]:
dvar = da.from_array(var,chunks=(chunk))
dvar

depth = {0:1,1:1}
boundary={0:'periodic',1:'periodic'}
odvar = da.overlap.overlap(dvar,depth=depth,boundary=boundary)

In [ ]:
%%time
for n in range(100):
    var = roll(var)

In [ ]:
%%time
for n in range(100):
    dvar = droll(dvar)
    
dvar.compute()

In [ ]:
%%time
for n in range(100):
    odvar = droll(odvar)

odvar.compute()

In [ ]:
%%time
y.compute()

In [ ]:
client = dd.Client(n_workers=8)

In [ ]:
@delayed
def inc(x):
    sleep(1)
    return x+1

@delayed
def add(x,y):
    sleep(1)
    return x+y

In [ ]:
%%time
x = inc(1)
y = inc(2)
z = add(x,y)

In [ ]:
%%time
z.compute()

In [ ]:
data = [1, 2, 3, 4, 5, 6, 7, 8]

In [ ]:
%%time
# Sequential code

results = []
for x in data:
    y = inc(x)
    results.append(y)

total = sum(results)

In [ ]:
total

In [ ]:
%%time
results = []

for x in data:
    y = delayed(inc)(x)
    results.append(y)

total = delayed(sum)(results)
print("Before computing:", total)  # Let's see what type of thing total is
result = total.compute()
print("After computing :", result)  # After it's computed
